In [1]:
import numpy as np
import cv2
import os 
from matplotlib import pyplot as plt
%matplotlib inline
from PIL import Image
from IPython import display

class VideoCamera(object):
    def __init__(self,index=0):
        self.video = cv2.VideoCapture(index)
        self.index=index
        print(self.video.isOpened())
        
    def __del__(self):
        self.video.release()
        
    def get_frame(self,in_grayscale=False):
        _,frame = self.video.read()
        if in_grayscale:
            frame = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        return frame  
    
def normalize_intensity(images):
    images_norm = []
    for image in images:
        is_color = len(image.shape) == 3
        if is_color:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        images_norm.append(cv2.equalizeHist(image))
    return images_norm 

In [2]:
class FaceDetector(object):
    def __init__(self, xml_path):
        self.classifier = cv2.CascadeClassifier(xml_path)
        
    def detect(self, image ,biggest_only=True):
        
        faces_coord = self.classifier.detectMultiScaler(image,1.3,5)
        return faces_coord

In [3]:
def resize(images, size = (50,50)):
    images_norm = []
    for image in images:
        if image.shape < size:
            image_norm = cv2.resize(image,size,
                                   interpolation = cv2.INTER_AREA)
        else:
            image_norm = cv2.resize(image,size,
                                   interpolation = cv2.INTER_CUBIC)
        images_norm.append(image_norm)
    return images_norm    

In [4]:
def cut_faces(image,faces_coord):
    faces = []
    
    for (x,y,w,h) in faces_coord:
        w_rm = int(0.2 * w/2)
        faces.append(image[y: y+h,x + w_rm:x + w - w_rm])
    return faces    

In [ ]:
def normalize_faces(frame,faces_coord):
    faces = cut_faces(frame,faces_coord)
    faces = normalize_intensity(faces)
    faces = resize(faces)
    return faces

def draw_rectangle(image, coords):
    for ( x, y, w, h) in coords:
        w_rm = int(0.2* w/2)
        cv2.rectangle(image, (x + w_rm,y), (x+ w - w_rm,y+h),
                             (150,150,0),8)
        



In [ ]:
detector = cv2.CascadeClassifier('/Users/spyroschouliaras/Downloads/frontalFace10/haarcascade_frontalface_default.xml')

webcam = VideoCamera()
folder = "/users/spyroschouliaras/Desktop/people/" + input('Person: ').lower()
cv2.namedWindow("PyData Tutorial",cv2.WINDOW_AUTOSIZE)
if not os.path.exists(folder):
    os.mkdir(folder)
    counter =0
    timer = 0
    while counter < 10:
        frame = webcam.get_frame()
        faces_coord = detector.detectMultiScale(frame,1.3,5)
        if len(faces_coord) and timer % 700 == 50:
            faces = normalize_faces(frame, faces_coord)
            cv2.imwrite(folder + '/' + str(counter) + '.jpg',faces[0])
            plt.imshow(faces[0])
            display.clear_output(wait = True)
            counter +=1
        draw_rectangle(frame,faces_coord)
        cv2.imshow("Pydata Tutorial",frame)
        cv2.waitKey(50)
        timer +=50
    cv2.destroyAllWindows()
else:
    print("this name already exists")

True
Person: sd
